In [ ]:

import requests
from datetime import datetime, timezone 
from Classes import Document, Author, HackerNewsDocument, TheGuardianDocument
from bs4 import BeautifulSoup
from Corpus import Corpus, DocumentFactory
from scipy.sparse import csr_matrix
from SearchEngine import SearchEngine
import pickle

#--------------Définition des variables
#variable pour stocker les documents à l'état 'brut'
collection = []
#Nombre d'articles à récupérer
nbDoc = 30
query = "war"
api_key_guardian = "265a16e3-294c-4c62-ae88-a274906a6333"


def search_query(texte,mots_cles) : 
    #Vérifie si un des mots clés est présent dans le texte ou le titre
    return any(mot.lower() in texte.lower() for mot in mots_cles)

#Fonction qui permet d'extraire le texte d'un article à partir de son url ou son auteur (pour The Guardian)  
#La balise HTML <a> contenant l'auteur a est propre à The Guardian  
def extraire_text_url_ou_auteur(url,txt_ou_auteur) : 
    try:
        #On récupère la page html puis on vérifie si la requête a réussi
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        if txt_ou_auteur == 0 : # texte
            #On extrait uniquement les balises <p> présentes dans le body
            body = soup.body
            if body is None:
                return "Texte non disponible" #Car pas de body
            
            paragraphes = body.find_all("p")    #Texte à partir du body
            texte = "\n".join(p.get_text(strip=True) for p in paragraphes)
            if(texte.strip) : 
                return texte
            else : 
                return "Texte non disponible"
            
        else : # auteur
            auteur_tag = soup.find("a", {"rel": "author", "data-link-name": "auto tag link"})
            if auteur_tag:
                return auteur_tag.get_text(strip=True)
            else:
                return "Auteur non trouvé"
        
    except Exception as e :
        return "Texte non disponible" if txt_ou_auteur==0 else f"Erreur lors de l'extraction de l'auteur: {e}"

#-----------------------WebScrapping avec Hacker News API
used_urls = set()
def add_doc_HackerNews(collection,query,nbDoc) :
    nbCount =0
    url = "https://hacker-news.firebaseio.com/v0/beststories.json"
    response = requests.get(url)
    if response.status_code !=200 : 
        raise Exception(f"Aucun texte provenant de HackerNews ne correspond à la recherche {query}")
    
    top_stories = response.json()
    for id in top_stories[:1000]: 
        if nbCount >= nbDoc:
            break

        url = f"https://hacker-news.firebaseio.com/v0/item/{id}.json"
        
        data = requests.get(url).json()
        titre = data.get("title", "No title")
        #Difficile de trouver l'auteur, on prendra celui de la story (les pages ne sont pas structurés de la même manière)
        auteur = data.get("by", "Unknown")
        timestamp = data.get("time", 0)
        date = datetime.fromtimestamp(timestamp, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
        article_url = data.get("url")
        score = data.get("score", 0)

        if article_url not in used_urls:
            texte = extraire_text_url_ou_auteur(article_url,0)
            if texte != "Texte non disponible" :
                if search_query(titre, query) or search_query(texte, query):
                    doc = DocumentFactory.creerDoc('HackerNews', titre, auteur, date, article_url, texte, score)
                    collection.append(doc)
                    nbCount +=1
                    used_urls.add(article_url)
    return collection

#-----------------------WebScrapping avec The Guardian API
def add_doc_Guardian(collection, query, nbDoc, api_key):
    nbCount = 0
    url = f"https://content.guardianapis.com/search?q={query}&page-size={nbDoc}&api-key={api_key}"
    response = requests.get(url)
    #Renvoie une exception si aucun article n'a été trouvé 
    if response.status_code !=200 : 
        raise Exception(f"Aucun article provenant de The Guardian ne correspond à la recherche {query}")
    
    data = response.json()
    for article in data["response"]["results"]:
        if nbCount >= nbDoc:
            break
        titre = article["webTitle"]
        article_url = article["webUrl"]
        try :  
            auteur = extraire_text_url_ou_auteur(article_url,1)
        except : 
            auteur = "The Guardian"
        release_date_str = article.get("firstPublicationDate", "")
        release_date = None     #Il faut instancier
        if release_date_str:
            release_date = datetime.strptime(release_date_str, "%Y-%m-%dT%H:%M:%S.%fZ")  
        
        # Dernière date de mise à jour (on récupère sous forme de string avant de la convertir en date)
        last_maj_str = article.get("lastModified", "")
        last_maj = None         #Il faut instancier
        if last_maj_str:
            last_maj = datetime.strptime(last_maj_str, "%Y-%m-%dT%H:%M:%S.%fZ") 

        if article_url not in used_urls:
            texte = extraire_text_url_ou_auteur(article_url,0)
            if texte != "Texte non disponible":
                doc = DocumentFactory.creerDoc('The_Guardian', titre, auteur, last_maj, article_url, texte, release_date)
                collection.append(doc)
                nbCount += 1
                used_urls.add(article_url)

    return collection

#-------------------RECUPERATION DES DONNEES
add_doc_HackerNews(collection,query,nbDoc) 
add_doc_Guardian(collection,query, nbDoc, api_key_guardian)

print(f"nbCount : {len(collection)}")    


# Création de l'index de documents à partir de la collection
#Clé : un Id 
#Valeur : le titre du document
id2doc = {i: doc.titre for i, doc in enumerate(collection)}

authors = {}
aut2id = {}
num_auteurs_vus = 0

# Création de l'index des Auteurs à partir de la collection
#Clé : un Id (en fonction de la valeur de la variable num_auteurs_vus)
#Valeur : un objet de type Author
for doc in collection:
    if doc.auteur not in aut2id:
        num_auteurs_vus += 1
        authors[num_auteurs_vus] = Author(doc.auteur)
        aut2id[doc.auteur] = num_auteurs_vus
    authors[aut2id[doc.auteur]].add(doc.texte)

# Construction du corpus à partir des documents présents dans la collection
corpus = Corpus(f"Corpus {query}")
for doc in collection:
    corpus.add(doc)

#------------- CONSTRUCTION D'UN 2E CORPUS -------------------------------------
collection2 = []
query2 = "computer"
add_doc_HackerNews(collection2,query2,nbDoc) 
add_doc_Guardian(collection2,query2, nbDoc, api_key_guardian)

print(f"nbCount : {len(collection2)}")    


# Rempliisage de l'index de documents à partir des document de la 2e la collection
#Clé : un Id 
#Valeur : le titre du document
for i, doc in enumerate(collection2, start=len(id2doc)):
    id2doc[i] = doc.titre


# Remplissage de l'index des Auteurs à partir des auteurs de la 2e collection
#Clé : un Id (en fonction de la valeur de la variable num_auteurs_vus)
#Valeur : un objet de type Author
for doc in collection2:
    if doc.auteur not in aut2id:
        num_auteurs_vus += 1
        authors[num_auteurs_vus] = Author(doc.auteur)
        aut2id[doc.auteur] = num_auteurs_vus
    authors[aut2id[doc.auteur]].add(doc.texte)

# Construction du corpus à partir des documents présents dans la collection
corpus2 = Corpus(f"Corpus {query2}")
for doc in collection2:
    corpus2.add(doc)

# Ouverture d'un fichier, puis écriture avec pickle
with open(f"corpus_{query}.pkl", "wb") as f:
    pickle.dump(corpus, f)
with open(f"corpus_{query2}.pkl", "wb") as f:
    pickle.dump(corpus2, f)

In [ ]:
from SearchEngine import SearchEngine
#Création du moteur de recherche
search_engine = SearchEngine(corpus)
results = search_engine.search(query)
search_engine2 = SearchEngine(corpus2)
results2 = search_engine2.search(query2)

search_engine.corpus.stats()
search_engine2.corpus.stats()

In [12]:
import ipywidgets as wg
from IPython.display import display, HTML
from Classes import Document, Author, HackerNewsDocument, TheGuardianDocument
from SearchEngine import SearchEngine
import pandas as pd
import pickle
import contextlib
import os

#Chargement du corpus
query = "war"
query2 = "computer"

with open(f"corpus_{query}.pkl", "rb") as f:
    corpus = pickle.load(f)
with open(f"corpus_{query2}.pkl", "rb") as f:
    corpus2 = pickle.load(f)

#============================ INITIALISATION DES VARIABLES =============================================
# Création de l'index de documents à partir du premier corpus
id2doc = {i: doc.titre for i, doc in enumerate(corpus.id2doc.values())}

authors = {}
aut2id = {}
num_auteurs_vus = 0

# Création de l'index des Auteurs à partir du premier corpus
for doc in corpus.id2doc.values():
    if doc.auteur not in aut2id:
        num_auteurs_vus += 1
        authors[num_auteurs_vus] = Author(doc.auteur)
        aut2id[doc.auteur] = num_auteurs_vus
    authors[aut2id[doc.auteur]].add(doc.texte)

# Remplissage de l'index de documents à partir du deuxième corpus
for i, doc in enumerate(corpus2.id2doc.values(), start=len(id2doc)):
    id2doc[i] = doc.titre

# Remplissage de l'index des Auteurs à partir du deuxième corpus
for doc in corpus2.id2doc.values():
    if doc.auteur not in aut2id:
        num_auteurs_vus += 1
        authors[num_auteurs_vus] = Author(doc.auteur)
        aut2id[doc.auteur] = num_auteurs_vus
    authors[aut2id[doc.auteur]].add(doc.texte)


#============================ INTERFACE GRAPHIQUE =============================================

label = wg.HTML(
    "<h2 style='color: lightpink; text-align: center;'>🕵🏽‍♀️ Mon moteur de recherche 🔎</h2>"
)

#Radio Boutton pour le choix de la recherche (un ou deux corpus à utiliser)
choix_recherche = wg.RadioButtons(
    options=[query, f'{query} / {query2}'],
    disabled=False,
    layout=wg.Layout(
        display='flex',
        flex_flow='row wrap',  
        align_items='center',
        margin='0px 0px 10px 0px'
    ),
    style={'description_width': '150px'}
)

text_widget = wg.Text(
    placeholder="Entrez vos mots ici",
    layout=wg.Layout(width='300px')
)

text_input = wg.HBox([
    wg.HTML(
        "<b style='font-size: 16px; color: lightgreen;'>Mots clés :</b>"
    ),
    text_widget
])

#Slider pour le choix du nombre de documents à afficher et du type de documents choisis
slider = wg.HBox([
    wg.HTML(
        "<b style='font-size: 12px; color: grey;'>Nb de documents max. par thème :</b>"
    ),
    wg.IntSlider(
        value=5, 
        min=1, 
        max=20, 
        step=1,
        layout=wg.Layout(width='150px')  
    ),
    wg.HTML(
        "<b style='font-size: 12px; color: grey; margin-left: 20px;'>Type :</b>"
    ),
    wg.Dropdown(
        options=["Tout"] + ["The_Guardian", "HackerNews"],  
        value="Tout",
        layout=wg.Layout(width='120px')
    )
])
    
slider_2_corpus = wg.HBox([
    wg.RadioButtons(
        options=['Mots clés', 'Auteurs'],
        description='Par :',
        layout=wg.Layout(width='200px'),
    ),
    wg.HBox([
        wg.Button( 
            icon="check",
            style={'button_color': '#FFDAB9'},
            layout=wg.Layout(width='30px', margin='10px 5px 0px 0px')
        ),
        wg.Button(
            icon="undo",
            style={'button_color': '#FFDAB9'},
            layout=wg.Layout(width='30px', margin='10px 0px 0px 5px')
        )
    ]),
], layout=wg.Layout(display='none'))

#Liste à choix multiples pour les auteurs, par défaut, on affiche les auteurs présents dans les deux corpus 
select_auteurs = wg.SelectMultiple(
    options=aut2id,
    description='Auteurs :',
    layout=wg.Layout(width='400px', height='100px', display='none')
)


#Radio Boutton pour le choix de la présence ou non des auteurs dans les recherches
choix_corpus_auteur = wg.RadioButtons(
    options=['Les deux', f'Thème {query}', f'Thème {query2}'],
    description='Présence des auteurs :',
    layout=wg.Layout(
        display='none',
        flex_flow='row wrap',  
        align_items='center',
        margin='0px'  
    ),
    style={'description_width': '150px'}
)

search_button = wg.Button(
    description="Rechercher", 
    icon="search",
    style={'button_color': '#FFDAB9'}
)

output = wg.Output()

#Fonction qui permet de désactiver les widgets du premier slider
def disable_slider() :
    slider.children[0].disabled = True
    slider.children[1].disabled = True
    slider.children[2].disabled = True
    slider.children[3].disabled = True

#Fonction qui permet d'activer les widgets du premier slider
def enable_slider() :
    slider.children[0].disabled = False
    slider.children[1].disabled = False
    slider.children[2].disabled = False
    slider.children[3].disabled = False 

#Fonction qui permet de désactiver les widgets du deuxième slider (si l'on sélectionne l'option "Deux corpus")
def disable_slider_2_corpus() :
    slider_2_corpus.children[0].disabled = True
    slider_2_corpus.children[1].children[0].disabled = True

#Fonction qui permet d'activer les widgets du deuxième slider (si l'on sélectionne l'option "Deux corpus")
def enable_slider_2_corpus() :
    slider_2_corpus.children[0].disabled = False
    slider_2_corpus.children[1].children[0].disabled = False


#Fonction qui permet de changer la disponibilité widgets quand l'utilisateur revient en arrière avec le bouton (cf. slider_2_corpus)
def changement_choix(b) : 
    choix_recherche.disabled = False
    disable_slider()
    enable_slider_2_corpus()
    search_button.disabled = True
    text_widget.disabled = True
    choix_corpus_auteur.layout.display = 'none'
    select_auteurs.layout.display = 'none'
    slider_2_corpus.children[1].children[1].disabled = True
slider_2_corpus.children[1].children[1].on_click(changement_choix)


#Selon le choix de l'utilisateur, on affiche, active ou non certains les widgets
#Réagit en fonction du choix de recherche (un ou deux corpus)
def on_choix_recherche_change(change):
    if change['new'] == query:
        slider_2_corpus.layout.display ='none'
        enable_slider()
        text_widget.disabled = False
        search_button.disabled = False
    else:
        slider_2_corpus.layout.display ='flex'
        disable_slider()
        text_widget.disabled = True
        search_button.disabled = True
    slider_2_corpus.children[1].children[1].disabled = True
choix_recherche.observe(on_choix_recherche_change, names='value')


#Quand l'utilisateur clique sur le bouton de validation, on affiche et active / désactive les widgets correspondants
def click_type_recherche(b):
    if slider_2_corpus.children[0].value == 'Mots clés':
        select_auteurs.layout.display = 'none'
        choix_corpus_auteur.layout.display = 'none'
        text_widget.disabled = False
    else:
        select_auteurs.layout.display = 'flex'
        choix_corpus_auteur.layout.display = 'flex'
        text_widget.disabled = True
        
    choix_recherche.disabled = True
    slider_2_corpus.children[1].children[1].disabled = False
    disable_slider_2_corpus()
    enable_slider()
    search_button.disabled = False
    
slider_2_corpus.children[1].children[0].on_click(click_type_recherche)



#=============================================== GESTION DES AUTEURS ===============================================
#Le code ci-dessous permet de gérer les auteurs en fonction du type de document et du corpus choisi par l'utilisateur (gère deux évenements en simultanée)

#=============================== Variables pour stocker la liste des auteurs présents dans les deux corpus =========
#Nécessité de définir des variables globales pour pouvoir les utiliser dans les fonctions
global authors_data, current_liste_author,liste_authors_corpus1,liste_authors_corpus2,liste_author_both,current_liste_choix_corpus,current_liste_choix_type
authors_data = {
    "Tout": list(set(corpus.get_name_authors_by_type()) | set(corpus2.get_name_authors_by_type())),
    "The_Guardian": list(set(corpus.get_name_authors_by_type('The_Guardian')) | set(corpus2.get_name_authors_by_type('The_Guardian'))),
    "HackerNews": list(set(corpus.get_name_authors_by_type('HackerNews')) | set(corpus2.get_name_authors_by_type('HackerNews')))
}
#On récupère les auteurs présents dans les deux corpus
liste_authors_corpus1 = corpus.get_name_authors_by_type()
liste_authors_corpus2 = corpus2.get_name_authors_by_type()
liste_author_both = list(set(liste_authors_corpus1) & set(liste_authors_corpus2))
#Variables pour retrouver les auteurs en fonction du choix de l'utilisateur (type de document et corpus)
current_liste_choix_corpus = liste_author_both
current_liste_choix_type = authors_data['Tout']
select_auteurs.options = liste_author_both

    
#Fonction qui permet de changer la liste des auteurs en fonction du type de document choisit
#Affiche dynamiquement dans le selecteur d'auteurs (select_auteurs)
def on_choix_type_doc_for_authors(change):
    global authors_data, current_liste_choix_corpus, current_liste_choix_type,select_auteurs
    if change['new'] == 'The_Guardian':
        current_liste_choix_type = authors_data['The_Guardian']
    elif change['new'] == 'HackerNews':
        current_liste_choix_type = authors_data['HackerNews']
    else :
        current_liste_choix_type = authors_data['Tout']
    select_auteurs.options = list(set(current_liste_choix_corpus) & set(current_liste_choix_type))
    return current_liste_choix_type
#Appel de la fonction à chaque changement de type de document
slider.children[3].observe(on_choix_type_doc_for_authors, names='value')   

#Fonction qui permet de changer la liste des auteurs en fonction du choix de corpus
#Affiche dynamiquement dans le selecteur d'auteurs (select_auteurs)
def on_choix_recherche_num_corpus_for_authors(change) :
    global current_liste_choix_corpus, current_liste_choix_type, liste_authors_corpus1, liste_authors_corpus2, liste_author_both, select_auteurs
    if change['new'] == 'Les deux':
        current_liste_choix_corpus = liste_author_both
    elif change['new'] == f'Thème {query}':
        current_liste_choix_corpus = liste_authors_corpus1
    else : 
        current_liste_choix_corpus = liste_authors_corpus2

    select_auteurs.options = list(set(current_liste_choix_corpus) & set(current_liste_choix_type))
    return current_liste_choix_corpus

choix_corpus_auteur.observe(on_choix_recherche_num_corpus_for_authors, names='value')

#======================================= FIN DE LA GESTION DES AUTEURS ============================================

#=======================
#Fonction qui permet de gérer les événements liés au clic sur le bouton de recherche
def clique_bouton(b) :
    query = text_widget.value
    num_docs = slider.children[1].value
    type_choisi = slider.children[3].value
    auteurs = select_auteurs.value
    with output :
        output.clear_output()
        if(choix_recherche.value == query) :
            if not query.strip() :
                display(HTML("<h3>Entrez des mots clés pour effectuer une recherche</h3>"))
                return
            search_corpus(corpus, query, num_docs, type_choisi)
        elif(choix_recherche.value == f'{query} / {query2}') :
            if slider_2_corpus.children[0].value == 'Auteurs' :
                #Si on fait une recherche en fonction du nom des auteurs 
                #Pas besoin de mots clés
                if auteurs == () :
                    display(HTML("<h3>Choisissez un ou plusieurs auteurs pour effectuer une recherche</h3>"))
                    return
                else :
                    res1 = corpus.get_doc_by_authors(auteurs)
                    res2 = corpus2.get_doc_by_authors(auteurs)
                    res = pd.concat([res1, res2])
                    #Supprimer les doublons en fonction du nom du document (Titre)
                    res = res.drop_duplicates(subset=['Titre'])
                display_results(res, num_docs)
            else : 
                #Recherche par mots clés
                if not query.strip() :
                    display(HTML("<h3>Entrez des mots clés pour effectuer une recherche</h3>"))
                    return
                compare_corpus(corpus, corpus2, query, num_docs, type_choisi)
    
search_button.on_click(clique_bouton)

#On ajoute les widgets à l'interface
interface = wg.VBox([label,choix_recherche, slider_2_corpus, text_input, slider,choix_corpus_auteur, select_auteurs, search_button, output],
        layout=wg.Layout(justify_content="center", 
        align_items="center",         
        padding="20px"))
display(interface)

#=================== Fonctions pour afficher les résultats de la recherche ========================

def display_results(result_generator1, max_docs=100):
    # on construit le code html 
    results_html = "<br>"
    for i, excerpt in result_generator1.iterrows():
        if i >= max_docs:  # limiter le nombre de résultats
            break
        # hypertextualiser (?) le lien pour accèder au document sur sa page web
        url = excerpt['URL'] if excerpt['URL'] != 'Non disponible' else None 
        type_doc = excerpt['Type']
        hyperlien = f"<a href='{url}' target='_blank' style='color: darkred;'>Accès au document sur {type_doc}</a>" if url else "Non disponible"
        results_html += (
            f"<div style='margin-bottom: 20px;'>"
            f"<b style='color: darkblue;'>{excerpt['Titre']}</b><br>"
            f"<b style='color: black;'>Auteur : </b>{excerpt['Auteur']}<br>"
            f"<b style='color: darkgreen;'>Extrait :</b> <b>{excerpt['Extrait']}</b><br>"
            f"{hyperlien}<br>"
            f"</div>"
        )
        
    display(HTML(results_html))


#======================================= FONCTIONS DE RECHERCHE ========================================
#Pour une recherche sur corpus
def search_corpus(corpus, query, max_docs, filtre_type_doc):
    search_engine = SearchEngine(corpus)
    with output:
        output.clear_output() # effacer la sortie précédente
        try:
            with contextlib.redirect_stdout(open(os.devnull, 'w')):
                result_generator = search_engine.search(query)
                
            if result_generator.empty:
                display(HTML(f"<h3>Aucun résultat trouvé pour la requête : '{query}'</h3>"))
                return
            # on vérifie si l'utilisateur a utilisé un filtre
            if filtre_type_doc != 'Tout' : 
                result_generator = result_generator[result_generator["Type"] == filtre_type_doc]
            display_results(result_generator, max_docs)
        except Exception as e:
            print("Erreur lors de la recherche :", str(e))
            return


#Pour une recherche sur deux corpus
def compare_corpus(corpus1, corpus2, query, num_docs, filtre_type_doc):
    with output:
        output.clear_output() # effacer la sortie précédente
        search_engine1 = SearchEngine(corpus1)
        search_engine2 = SearchEngine(corpus2)
        try :
            with contextlib.redirect_stdout(open(os.devnull, 'w')):
                result_generator1 = search_engine1.search(query)
                result_generator2 = search_engine2.search(query)
                
            if result_generator1.empty and result_generator2.empty:
                display(HTML(f"<h3>Aucun résultat trouvé pour la requête : '{query}'</h3>"))
                return
            output.clear_output() # effacer la sortie précédente
            # on vérifie si l'utilisateur a utilisé un filtre pour le type
            if filtre_type_doc != 'Tout' : 
                result_generator1 = result_generator1[result_generator1["Type"] == filtre_type_doc]
                result_generator2 = result_generator2[result_generator2["Type"] == filtre_type_doc]
            
            display(HTML("<h3><u>Résultats pour War :</h3><u>"))
            if result_generator1.empty:
                display(HTML(f"<p>Aucun résultat trouvé pour la requête : '{query}' pour le thème War</p>"))
            else :
                display_results(result_generator1, num_docs)
            display(HTML("<h3><u>Résultats pour Computer :</h3><u>"))
            if result_generator2.empty:
                display(HTML(f"<p>Aucun résultat trouvé pour la requête : '{query}' pour le thème Computer</p>"))
            else :
                display_results(result_generator2, num_docs)
        except Exception as e:
            print("Erreur lors de la recherche :", str(e))
            return
#======================================= FIN DES FONCTIONS DE RECHERCHE ========================================
